In [ ]:
import numpy as np
import torch
import torch.nn as nn
import sys

sys.path.append("../")
from vi_rnn.vae import VAE
from vi_rnn.train import train_VAE
from torch.utils.data import DataLoader
from vi_rnn.datasets import RDM, RDM_Teacher
from py_rnn.model import RNN, predict
from py_rnn.train import train_rnn
from py_rnn.train import save_rnn, load_rnn
from vi_rnn.utils import *
import matplotlib.pyplot as plt
from vi_rnn.saving import save_model, load_model
from py_rnn.default_params import get_default_params

from vi_rnn.utils import *
import matplotlib as mpl
from fixed_points.find_fixed_points_analytic import find_fixed_points_analytic
from fixed_points.stability import PL_Jacobian
from matplotlib.markers import MarkerStyle
from matplotlib.colors import colorConverter as cc

%matplotlib inline

In [ ]:
data_dir = "../data/student_teacher/"  # store inferred model
model_dir = "../models/students/"  # store teacher RNN
cuda = True  # toggle if GPU is available

In [ ]:
# load teacher
rnn_reach, model_params, task_params_teacher, training_params_teacher = load_rnn(
    data_dir + "rdm_rnn"
)
U, V, B = extract_orth_basis_rnn(rnn_reach)
I = rnn_reach.rnn.w_inp.detach()

# load student
vae, vae_params, task_params, training_params = load_model(
    name=model_dir + "rdm_student_new"
)
vae = orthogonalise_network(vae)
tau, pV, pU, pB, pI = get_loadings(vae)

In [ ]:
task_params

In [ ]:
# make data / task
task_params_teacher["response_dur"] = 50
batch_size = 12
task_params["n_trials"] = batch_size
task_params['sim_v']=True
task = RDM_Teacher(task_params, task_params_teacher, U, V, B, I, decay=0.9)
ratesGT = task.data
latent_codeGT = task.latents

# run the student RNN
with torch.no_grad():
    z0 = torch.zeros(batch_size, 1)
    stim = task.stim
    dur = task.dur
    Z, v = vae.rnn.get_latent_time_series(
        time_steps=dur, z0=z0, u=stim, noise_scale=1, sim_v=True
    )
    Zn = Z.cpu().detach().numpy()[:, :, :, 0]
    data_gen = (
        vae.rnn.get_observation(Z, v=v, noise_scale=0)
        .cpu()
        .detach()
        .numpy()[:, :, :, 0]
    )

In [ ]:
# Get all fixed points of teacher RNN
D_list, D_inds, z_listGT, n_inverses = find_fixed_points_analytic(
    np.ones(1) * 0.9, V.numpy(), U.numpy(), 0, -B.numpy()
)

# Get all fixed points of student RNN
D_list, D_inds, z_list, n_inverses = find_fixed_points_analytic(
    np.array([tau]), pV, pU, 0, pB
)

In [ ]:
# make plot

with mpl.rc_context(fname="matplotlibrc"):

    colors = [
        "darkred",
        "firebrick",
        "tomato",
        "darkturquoise",
        "teal",
        "darkslategrey",
    ]
    fig, ax = plt.subplots(1, 3, figsize=(5, 1))
    for i in range(6):
        ax[2].plot(Z[i, 0, :, 0].T, v[i, 0, :, 0].T, color=colors[i % 6])
        latent_code = task.latents[i]
        task_v = task.v[i]
        ax[1].plot(latent_code[0], task_v[0], color=colors[i % 6])
        ax[0].plot(task.stim[i, 0], color=colors[i % 6])
    ax[1].set_xlabel(r"$z$")
    ax[1].set_ylabel(r"$\tilde{s}$")
    ax[0].set_ylabel(r"$s$")
    ax[0].set_xlabel("timesteps")
    ax[2].set_xlabel(r"$z$")
    ax[1].set_title("teacher")
    ax[2].set_title("student")
    # marker style
    dot_s = 10
    dot_z = 100
    dot_ew = 0.4
    dot_s_st = 20
    dot_fill = "gainsboro"

    # Calculate stability of fixed points and plots
    for z in z_list:
        e, vec = np.linalg.eig(PL_Jacobian(pV, pU, -pB, np.diag(np.ones(1) * tau), z))
        if e[0] > 1:
            ax[2].scatter(
                z[0],
                0,
                s=dot_s,
                c=dot_fill,
                edgecolor="black",
                lw=dot_ew,
                marker=MarkerStyle("o"),
                zorder=dot_z,
            )
        else:
            ax[2].scatter(
                z[0],
                0,
                s=dot_s,
                c="black",
                edgecolor="black",
                lw=dot_ew,
                marker=MarkerStyle("o"),
                zorder=dot_z,
            )

    for z in z_listGT:
        e, vec = np.linalg.eig(
            PL_Jacobian(V.numpy(), U.numpy(), B.numpy(), np.diag(np.ones(1) * 0.9), z)
        )
        if e[0] > 1:
            ax[1].scatter(
                z[0],
                0,
                s=dot_s,
                c=dot_fill,
                edgecolor="black",
                lw=dot_ew,
                marker=MarkerStyle("o"),
                zorder=dot_z,
            )
        else:
            ax[1].scatter(
                z[0],
                0,
                s=dot_s,
                c="black",
                edgecolor="black",
                lw=dot_ew,
                marker=MarkerStyle("o"),
                zorder=dot_z,
            )

    ax[1].set_xlim(-15, 15)
    ax[1].set_ylim(-0.6, 0.6)
    ax[2].set_xlim(-15, 15)
    ax[2].set_ylim(-0.6, 0.6)
    ax[2].set_ylim(-0.6, 0.6)
    ax[0].set_box_aspect(1)
    ax[1].set_box_aspect(1)
    ax[2].set_box_aspect(1)
    ax[2].set_yticklabels([])
    ax[0].set_yticks([-0.5, 0, 0.5])
    plt.subplots_adjust(wspace=0.5)

    ax[0].set_yticklabels(["-.5", "0", ".5"])
    ax[1].set_yticks([-0.5, 0, 0.5])
    ax[1].set_yticklabels(["-.5", "0", ".5"])
    ax[0].set_xlim(0, 100)
    ax[0].set_title("inputs")
    plt.savefig("../figures/inputs.pdf")
    plt.savefig("../figures/inputs.png")